# Import libraries

In [ ]:
!pip install openai cohere tiktoken typing-extensions==4.5.0

from google.colab import userdata

import inspect
import os
import base64
import numpy as np
import pandas as pd

from openai import OpenAI

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Authentication

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

# Import dataset

In [ ]:
total_defense_memes_test = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_test.csv')

# Define a function to generate text using the `gpt-4-vision-preview` model

In [ ]:
# Function to encode the image. via
# https://platform.openai.com/docs/guides/vision
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# Prevent rate limit errors. via
# https://github.com/openai/openai-cookbook/blob/main/examples/
# How_to_handle_rate_limits.ipynb
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
# Call the OPENAI API. via
# https://platform.openai.com/docs/guides/vision
def caption(image):
    completion = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
        "role": "user",
        "content": [
            {"type": "text", "text": 'What does the meme show, in particular text and entities? Describe in english:'},
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encode_image(image)}",
            },
            },
        ],
        }
    ],
    max_tokens=4096,
    temperature=0,
    seed=1,
    )

    return completion.choices[0].message.content, completion.usage.prompt_tokens, completion.usage.completion_tokens

# Call the `caption` function and save inferences

In [ ]:
%%time
df_temp = total_defense_memes_test.copy(deep=True)
# Create new columns by applying function. via
# https://stackoverflow.com/a/52363890
df_temp[['caption', 'prompt_tokens_caption', 'completion_tokens_caption']] = df_temp.apply(lambda row: caption(row.image), axis='columns', result_type='expand')
df_temp.to_csv('/content/drive/MyDrive/stance_detection_datasets_GPT/total_defense_memes/total_defense_memes_test_caption.csv', index=False)

CPU times: user 48.7 s, sys: 5.76 s, total: 54.5 s
Wall time: 2h 4min 38s
